In [38]:
from chicktimer import ChickTimer

# 3 sec gap == 20BPM          ( marks start of CT sequence and after each 5 beep seperator )
# 3.8 sec gap == 15.789BPM    ( after each number in number pairs )
# 1.3 sec gap == 46.1538BPM   ( between each beep of 5 beep seperators )
# 0.8 sec gap == 75.00BPM     ( between each beep of data beeps )

class BeepStateMachine:

    gap_beep_rate_3sec: float = 20.00
    gap_beep_rate_3_8sec: float = 15.789
    gap_beep_rate_1_3sec: float = 46.153
    gap_beep_rate_0_8: float = 75.00

    number1_count: int
    number2_count: int
    seperator_count: int
    pair_count: int
    

    def __init__(self, inital_state:str = "BACKGROUND"):
        self.state = inital_state
        self.number1_count = 0
        self.number2_count = 0
        self.seperator_count = 0
        self.pair_count = 0
        self.ct = ChickTimer()
        

    def process_input(self, BPM: float) -> None|ChickTimer:
        if self.state == "BACKGROUND":
            if any(abs(BPM-background_beep_rate) < 0.5 for background_beep_rate in [80, 46, 30] ):
                # background beep rate, do nothing, return nothing, exit
                return
            if (abs(BPM - self.gap_beep_rate_3sec) < 0.5):
                # 3 secon pause encountered - indicates first set of digits
                self.state = "NUMBER1"
                return
    
        if self.state == "NUMBER1":
            # check expected BPM and if so count and increment
            if (abs(BPM - self.gap_beep_rate_0_8) < 0.5 ):
                self.number1_count += 1
                return
            # if BPM is 15.78 - exit as that was last beep of the set
            if (abs(BPM - self.gap_beep_rate_3_8sec ) < 0.5):
                self.state = "NUMBER2"
                return # this return needs to exit both loops?
       
        if self.state == "NUMBER2":
            if (abs(BPM - self.gap_beep_rate_0_8) < 0.5 ):
                self.number2_count += 1
                return
            # if BPM is 15.78 - exit as last beep was last beep of that set
            if (abs(BPM - self.gap_beep_rate_3_8sec ) < 0.5):
                self.ct.setField(self.pair_count, int(f"{self.number1_count}{self.number2_count}" ) )
                self.number1_count = 0
                self.number2_count = 0
                self.state = "SEPERATOR"
                return
            
        if self.state == "SEPERATOR":
            if (abs(BPM - self.gap_beep_rate_1_3sec) < 0.5):
                self.seperator_count += 1
                return
            if (self.seperator_count == 5): # check for 5 beeps in seperator - should I also check that the gap is 3s?
                self.seperator_count = 0
                self.state = "NUMBER1"
                self.pair_count += 1
                return
        
        # Check we have 8 pairs of numbers and a 3 sec end pause
        if (self.pair_count == 7 and abs(BPM - self.gap_beep_rate_3sec) < 0.5):
            print(f"CT's have been recorded : {self.ct}")
            self.number1_count = 0
            self.number2_count = 0
            self.seperator_count = 0
            self.pair_count = 0
            self.state = "BACKGROUND"
            return
            
            
bsm = BeepStateMachine()
bsm.process_input(20) # 3 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(75) # 3
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(75) # 3
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(46.153) # 1
bsm.process_input(46.153) # 2 
bsm.process_input(46.153) # 3
bsm.process_input(46.153) # 4
bsm.process_input(46.153) # 5
bsm.process_input(20) # 3 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(46.153) # 1
bsm.process_input(46.153) # 2 
bsm.process_input(46.153) # 3
bsm.process_input(46.153) # 4
bsm.process_input(46.153) # 5
bsm.process_input(20) # 3 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(46.153) # 1
bsm.process_input(46.153) # 2 
bsm.process_input(46.153) # 3
bsm.process_input(46.153) # 4
bsm.process_input(46.153) # 5
bsm.process_input(20) # 3 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(75) # 3
bsm.process_input(75) # 4
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(75) # 3
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(46.153) # 1
bsm.process_input(46.153) # 2 
bsm.process_input(46.153) # 3
bsm.process_input(46.153) # 4
bsm.process_input(46.153) # 5
bsm.process_input(20) # 3 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(75) # 3
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(46.153) # 1
bsm.process_input(46.153) # 2 
bsm.process_input(46.153) # 3
bsm.process_input(46.153) # 4
bsm.process_input(46.153) # 5
bsm.process_input(20) # 3 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(75) # 3
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(46.153) # 1
bsm.process_input(46.153) # 2 
bsm.process_input(46.153) # 3
bsm.process_input(46.153) # 4
bsm.process_input(46.153) # 5
bsm.process_input(20) # 3 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(75) # 3
bsm.process_input(75) # 4
bsm.process_input(75) # 5
bsm.process_input(75) # 6
bsm.process_input(75) # 7
bsm.process_input(75) # 8
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(46.153) # 1
bsm.process_input(46.153) # 2 
bsm.process_input(46.153) # 3
bsm.process_input(46.153) # 4
bsm.process_input(46.153) # 5
bsm.process_input(20) # 3 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(75) # 3
bsm.process_input(75) # 4
bsm.process_input(75) # 5
bsm.process_input(15.78) # 3.8 sec seperator
bsm.process_input(75) # 1
bsm.process_input(75) # 2
bsm.process_input(75) # 3
bsm.process_input(75) # 4
bsm.process_input(75) # 5
bsm.process_input(75) # 6
bsm.process_input(75) # 7
bsm.process_input(15.78) # 3.8 or 3? sec seperator - CT BEEPS END WITH 3 SEC PAUSE









